# MNIST Train and Evaluate sample
## Multi-layer Perseptron

MNISTサンプルのMLP（多層パーセプトロン）バージョンです。

Adapted from https://github.com/keras-team/keras/blob/c45f48eaeab2a9c9a809488950f07d83abe18411/examples/mnist_mlp.py  
which is released under the MIT License, "Copyright (c) 2015, François Chollet."

Copyright 2022 RT Corporation  
SPDX-License-Identifier: [Apache-2.0](http://www.apache.org/licenses/LICENSE-2.0)

In [ ]:
# ライブラリ読み込み
import os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.optimizers import RMSprop

import matplotlib.pyplot as plt

In [ ]:
#
# 各種パラメータ

# Model fileの指定
model_file = "models/mnist_mlp.h5"

# 入力
img_width, img_height, img_depth = 28, 28, 1

# 出力
num_classes = 10

# 学習時のパラメータ
batch_size = 128
epochs = 20

In [ ]:
#
# MNISTのデータを読み込む
# ない場合は自動でダウンロードする
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#
# データを整形
# 画像を1次元配列化
input_shape = img_width * img_height
x_train = x_train.reshape(x_train.shape[0], input_shape)
x_train = x_train.astype('float32')
x_train /= 255
x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
x_test /= 255

print('x_train.shape: {}'.format(x_train.shape))
print('{} train samples'.format(x_train.shape[0]))
print('{} test samples'.format(x_test.shape[0]))

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


In [ ]:
#
# MLPモデルの取得
# 構築済みのモデルがある場合はそれを使う
# 指定したファイルやディレクトリがあるかを確認する
if os.path.exists(model_file):
    # あった場合はモデルを取り込む
    model = load_model(model_file)
    # そのモデルの構造を表示する
    model.summary()
else:
    # ない場合は、モデルを構築する
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=RMSprop(),
                metrics=['accuracy'])

    # モデルの構造を表示する
    model.summary()

In [ ]:
#
# 実際に学習を行う
history = model.fit(x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(x_test, y_test))

In [ ]:
#
# トレーニングの経過を表示

plt.figure(figsize=(12, 4))

# accuracy 正解率
plt.subplot(1, 2, 1)
plt.title('accuracy')
plt.plot(range(1,epochs+1), history.history['accuracy'], label='training')
plt.plot(range(1,epochs+1), history.history['val_accuracy'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# loss 損失関数
plt.subplot(1, 2, 2)
plt.title('loss')
plt.plot(range(1,epochs+1), history.history['loss'], label='training')
plt.plot(range(1,epochs+1), history.history['val_loss'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
print('batch size: {}, epochs: {}'.format(batch_size, epochs))

In [ ]:
#
# 学習結果を評価する
score = model.evaluate(x_test, y_test, verbose=0)

print('batch size: {}, epochs: {}'.format(batch_size, epochs))
print('Test loss:', score[0])
print('Test accuacy:', score[1])


In [ ]:
#
# モデルの保存
model.save(model_file)